In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from data_preprocessing import get_input, get_input_share_tokenizer

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Activation, Bidirectional, Dropout
from keras.layers.embeddings import Embedding

from tensorflow.keras import regularizers
from io import BytesIO

In [3]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units, l2_kernel, l2_recurrent, l2_activity, dropout):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim,
                                trainable = True, mask_zero = True)(X_input)
    
    X = LSTM(units = lstm_units, return_sequences = False,
            kernel_regularizer = regularizers.l2(l2_kernel),
            recurrent_regularizer = regularizers.l2(l2_recurrent),
            activity_regularizer = regularizers.l2(l2_activity))(embedding_layer)
    
    X = Dropout(rate = dropout)(X)
    X = Dense(units = 1)(X)
    X = Activation('sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [138]:
#model = create_model(10000, 500, 100, lstm_units = 128, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0.01, dropout = 0.5)
#model.summary()

In [139]:
def grid_search(X, dataset, epochs, models):
    
    Y = dataset.label.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)

    mean_dev_accuracy = np.zeros(np.size(models))
    
    i = 0
    for model in models:
    
        iterations = 5
        dev_accuracy_shuffle_split = np.zeros(iterations)
        shuffle = ShuffleSplit(n_splits = iterations, test_size = 0.2)
        
        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
        w = BytesIO()
        model.save_weights(w) # Save initial weights
        
        j = 0
        for train, dev in shuffle.split(X_train, y_train):
            Xtrain = X_train[train]
            Ytrain = y_train[train]
            Xdev = X_train[dev]
            Ydev = y_train[dev]

            model.fit(Xtrain, Ytrain, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

            loss, accuracy_val = model.evaluate(Xdev, Ydev) # Validate model
            dev_accuracy_shuffle_split[j] = accuracy_val

            model.load_weights(w) # Restore initial weights
            
            j += 1
              
        mean_dev_accuracy[i] = round(np.mean(dev_accuracy_shuffle_split), 3)
        std = round(np.std(dev_accuracy_shuffle_split), 3)
        print('Model ' + str(i) +' --> dev_acc: ' + str(mean_dev_accuracy[i]) + ' +- ' + str(std))
        i += 1
    
    # test
    best_model_index = np.argmax(mean_dev_accuracy)
    best_model = models[best_model_index]
    best_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile best model
    best_model.fit(X_train, y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit best model
    y_pred = best_model.predict(X_test) # Test best model
    y_pred = y_pred > 0.5 # Sigmoid activation function
    accuracy_test = accuracy_score(y_test, y_pred)
    
    return best_model_index, round(accuracy_test, 3)

In [81]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
5,Un paso más cerca de hacer los exámenes 'onlin...,0
6,UNAM REALIZARÁ PRUEBAS ANTIDOPING A ESTUDIANTE...,1
7,Niño de *NUMBER* años se prepara para entrar a...,0
8,*NUMBER* palabras que creíamos inaceptables y ...,0
9,LIMITARÁN EL TIEMPO DE EGRESO EN FILOSOFÍA Y S...,1


#### Models

In [82]:
vocabulary_length = 10000
max_length_sequence = 500
emb_dim = 100
language = 'spanish'
epochs = 7

In [83]:
X, df = get_input(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(2571, 500)


In [74]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 128, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)

model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1, dropout = 0)
model_4 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1, dropout = 0)
model_5 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 128, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1, dropout = 0)

models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)
models.append(model_4)
models.append(model_5)

In [76]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/7
1644/1644 [==============================] - 21s 13ms/step - loss: 1.6246 - accuracy: 0.5572
Epoch 2/7
1644/1644 [==============================] - 21s 13ms/step - loss: 0.9466 - accuracy: 0.8151
Epoch 3/7
1644/1644 [==============================] - 24s 15ms/step - loss: 0.4523 - accuracy: 0.9215
Epoch 4/7
1644/1644 [==============================] - 35s 21ms/step - loss: 0.2024 - accuracy: 0.9793
Epoch 5/7
1644/1644 [==============================] - 50s 30ms/step - loss: 0.1522 - accuracy: 0.9891
Epoch 6/7
1644/1644 [==============================] - 40s 24ms/step - loss: 0.0951 - accuracy: 0.9976
Epoch 7/7
412/412 [==============================] - 3s 8ms/step
Epoch 1/7
1644/1644 [==============================] - 30s 19ms/step - loss: 1.5104 - accuracy: 0.5213
Epoch 2/7
1644/1644 [==============================] - 31s 19ms/step - loss: 0.8818 - accuracy: 0.6849
Epoch 3/7
1644/1644 [==============================] - 30s 18ms/step - loss: 0.7615 - accuracy: 0.8583
Epoch 4/

412/412 [==============================] - 3s 7ms/step
Epoch 1/7
1644/1644 [==============================] - 23s 14ms/step - loss: 86.0888 - accuracy: 0.6107
Epoch 2/7
1644/1644 [==============================] - 24s 15ms/step - loss: 6.1190 - accuracy: 0.7962
Epoch 3/7
1644/1644 [==============================] - 26s 16ms/step - loss: 1.2572 - accuracy: 0.7731
Epoch 4/7
1644/1644 [==============================] - 25s 15ms/step - loss: 0.7893 - accuracy: 0.6606
Epoch 5/7
1644/1644 [==============================] - 25s 15ms/step - loss: 0.7127 - accuracy: 0.7652
Epoch 6/7
1644/1644 [==============================] - 24s 15ms/step - loss: 0.6927 - accuracy: 0.7987
Epoch 7/7
412/412 [==============================] - 2s 4ms/step
Epoch 1/7
1644/1644 [==============================] - 23s 14ms/step - loss: 77.4391 - accuracy: 0.6144
Epoch 2/7
1644/1644 [==============================] - 23s 14ms/step - loss: 3.6520 - accuracy: 0.6782
Epoch 3/7
1644/1644 [==============================] -

New Iterations

In [98]:
epochs = 7
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 2, l2_kernel = 0.5, l2_recurrent = 0.5, l2_activity = 0, dropout = 0.5)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 6, l2_kernel = 0, l2_recurrent = 0, l2_activity = 0, dropout = 0.5)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 8, l2_kernel = 0.1, l2_recurrent = 0.1, l2_activity = 0, dropout = 0.5)

models.append(model_0)
models.append(model_1)
models.append(model_2)

In [99]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/7
1644/1644 [==============================] - 19s 11ms/step - loss: 6.9912 - accuracy: 0.5383
Epoch 2/7
1644/1644 [==============================] - 17s 10ms/step - loss: 3.9686 - accuracy: 0.6296
Epoch 3/7
1644/1644 [==============================] - 18s 11ms/step - loss: 2.3924 - accuracy: 0.6679
Epoch 4/7
1644/1644 [==============================] - 19s 11ms/step - loss: 1.5926 - accuracy: 0.7117
Epoch 5/7
1644/1644 [==============================] - 20s 12ms/step - loss: 1.1927 - accuracy: 0.7111
Epoch 6/7
1644/1644 [==============================] - 21s 13ms/step - loss: 0.9883 - accuracy: 0.7932
Epoch 7/7
412/412 [==============================] - 3s 7ms/step
Epoch 1/7
1644/1644 [==============================] - 22s 13ms/step - loss: 6.2644 - accuracy: 0.5322
Epoch 2/7
1644/1644 [==============================] - 21s 13ms/step - loss: 2.6775 - accuracy: 0.6429
Epoch 3/7
1644/1644 [==============================] - 21s 13ms/step - loss: 1.5830 - accuracy: 0.6764
Epoch 4/

In [100]:
epochs = 25
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 8, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0, dropout = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 16, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0, dropout = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0, dropout = 0)

models.append(model_0)
models.append(model_1)
models.append(model_2)

In [101]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/25
1644/1644 [==============================] - 25s 15ms/step - loss: 41.5468 - accuracy: 0.5474
Epoch 2/25
1644/1644 [==============================] - 23s 14ms/step - loss: 19.7785 - accuracy: 0.7804
Epoch 3/25
1644/1644 [==============================] - 23s 14ms/step - loss: 9.3131 - accuracy: 0.8577
Epoch 4/25
1644/1644 [==============================] - 24s 14ms/step - loss: 4.5103 - accuracy: 0.7841
Epoch 5/25
1644/1644 [==============================] - 23s 14ms/step - loss: 2.3911 - accuracy: 0.6436
Epoch 6/25
1644/1644 [==============================] - 23s 14ms/step - loss: 1.4745 - accuracy: 0.6229
Epoch 7/25
1644/1644 [==============================] - 24s 15ms/step - loss: 1.0663 - accuracy: 0.8741
Epoch 8/25
1644/1644 [==============================] - 22s 13ms/step - loss: 0.8573 - accuracy: 0.9459
Epoch 9/25
1644/1644 [==============================] - 22s 14ms/step - loss: 0.6935 - accuracy: 0.9459
Epoch 10/25
1644/1644 [==============================] - 23s 1

1644/1644 [==============================] - 21s 13ms/step - loss: 1.0105 - accuracy: 0.5012
Epoch 4/25
1644/1644 [==============================] - 22s 13ms/step - loss: 0.7873 - accuracy: 0.5614
Epoch 5/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.7249 - accuracy: 0.8449
Epoch 6/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.7042 - accuracy: 0.7494
Epoch 7/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.6967 - accuracy: 0.5103
Epoch 8/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.6937 - accuracy: 0.7391
Epoch 9/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.6925 - accuracy: 0.7129
Epoch 10/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.6915 - accuracy: 0.7999
Epoch 11/25
1644/1644 [==============================] - 21s 13ms/step - loss: 0.6902 - accuracy: 0.5748
Epoch 12/25
1644/1644 [==============================] - 22s 13ms/step - 

1644/1644 [==============================] - 24s 14ms/step - loss: 0.6935 - accuracy: 0.5043
Epoch 6/25
1644/1644 [==============================] - 24s 14ms/step - loss: 0.6932 - accuracy: 0.5049
Epoch 7/25
1644/1644 [==============================] - 24s 14ms/step - loss: 0.6930 - accuracy: 0.5043
Epoch 8/25
1644/1644 [==============================] - 23s 14ms/step - loss: 0.6930 - accuracy: 0.5043
Epoch 9/25
1644/1644 [==============================] - 23s 14ms/step - loss: 0.6931 - accuracy: 0.5043
Epoch 10/25
1644/1644 [==============================] - 24s 14ms/step - loss: 0.6930 - accuracy: 0.5043
Epoch 11/25
1644/1644 [==============================] - 24s 14ms/step - loss: 0.6930 - accuracy: 0.5043
Epoch 12/25
1644/1644 [==============================] - 23s 14ms/step - loss: 0.6930 - accuracy: 0.5043
Epoch 13/25
1644/1644 [==============================] - 24s 15ms/step - loss: 0.6928 - accuracy: 0.5043
Epoch 14/25
1644/1644 [==============================] - 24s 14ms/step 

In [102]:
epochs = 10
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 16, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0.5)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0.5)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0.5)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 0.1, l2_recurrent = 0.1, l2_activity = 0, dropout = 0.5)


models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)

In [103]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/10
1644/1644 [==============================] - 22s 14ms/step - loss: 1.3477 - accuracy: 0.5444
Epoch 2/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.9531 - accuracy: 0.7798
Epoch 3/10
1644/1644 [==============================] - 22s 13ms/step - loss: 0.6278 - accuracy: 0.8674
Epoch 4/10
1644/1644 [==============================] - 22s 13ms/step - loss: 0.3125 - accuracy: 0.9611
Epoch 5/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.1889 - accuracy: 0.9903
Epoch 6/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.1389 - accuracy: 0.9976
Epoch 7/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.1143 - accuracy: 0.9964
Epoch 8/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.0981 - accuracy: 0.9988
Epoch 9/10
1644/1644 [==============================] - 21s 13ms/step - loss: 0.0798 - accuracy: 0.9988
Epoch 10/10
412/412 [==============================] - 3s 7ms/st

1644/1644 [==============================] - 25s 15ms/step - loss: 0.0829 - accuracy: 0.9988
Epoch 10/10
412/412 [==============================] - 2s 4ms/step
Model 2 --> dev_acc: 0.72 +- 0.027
Epoch 1/10
1644/1644 [==============================] - 27s 16ms/step - loss: 12.3394 - accuracy: 0.5073
Epoch 2/10
1644/1644 [==============================] - 26s 16ms/step - loss: 3.4400 - accuracy: 0.6356
Epoch 3/10
1644/1644 [==============================] - 25s 16ms/step - loss: 1.2553 - accuracy: 0.7220
Epoch 4/10
1644/1644 [==============================] - 25s 15ms/step - loss: 0.7657 - accuracy: 0.9179
Epoch 5/10
1644/1644 [==============================] - 26s 16ms/step - loss: 0.4831 - accuracy: 0.9197
Epoch 6/10
1644/1644 [==============================] - 25s 15ms/step - loss: 0.2688 - accuracy: 0.9799
Epoch 7/10
1644/1644 [==============================] - 25s 15ms/step - loss: 0.1859 - accuracy: 0.9951
Epoch 8/10
1644/1644 [==============================] - 25s 15ms/step - loss

#### Test Model with English Dataset

In [127]:
dataset = pd.read_csv('../data/Merged/english_dataset.csv')
dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
5,The number of cases of cops brutalizing and ki...,1
6,Donald Trump spent a good portion of his day a...,1
7,In the wake of yet another court decision that...,1
8,Many people have raised the alarm regarding th...,1
9,Just when you might have thought we d get a br...,1


In [140]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 4

In [141]:
X, df = get_input(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(51233, 1500)


In [142]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 2, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 4, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 8, l2_kernel = 0.1, l2_recurrent = 0.1, l2_activity = 0, dropout = 0)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 8, l2_kernel = 0, l2_recurrent = 0, l2_activity = 0, dropout = 0.5)

models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)

In [143]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/4
32788/32788 [==============================] - 1244s 38ms/step - loss: 0.3811 - accuracy: 0.9163
Epoch 2/4
32788/32788 [==============================] - 1299s 40ms/step - loss: 0.1924 - accuracy: 0.9507
Epoch 3/4
32788/32788 [==============================] - 1798s 55ms/step - loss: 0.1461 - accuracy: 0.9632
Epoch 4/4
8198/8198 [==============================] - 55s 7ms/step
Epoch 1/4
32788/32788 [==============================] - 1095s 33ms/step - loss: 0.5501 - accuracy: 0.8468
Epoch 2/4
32788/32788 [==============================] - 1083s 33ms/step - loss: 0.3032 - accuracy: 0.9117
Epoch 3/4
32788/32788 [==============================] - 1269s 39ms/step - loss: 0.2159 - accuracy: 0.9385
Epoch 4/4
8198/8198 [==============================] - 55s 7ms/step
Epoch 1/4
32788/32788 [==============================] - 1137s 35ms/step - loss: 0.4443 - accuracy: 0.8678
Epoch 2/4
32788/32788 [==============================] - 1223s 37ms/step - loss: 0.1858 - accuracy: 0.9467
Epoch 3/

#### Train Model with English Dataset and Evaluate with Translated Dataset

In [146]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
english_dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
5,The number of cases of cops brutalizing and ki...,1
6,Donald Trump spent a good portion of his day a...,1
7,In the wake of yet another court decision that...,1
8,Many people have raised the alarm regarding th...,1
9,Just when you might have thought we d get a br...,1


In [147]:
translated_dataset = pd.read_csv('../data/Merged/spanish_t_dataset.csv')
translated_dataset

,text,label
0,"RAE WILL INCLUDE THE WORD ""LADY"" IN THE SPANIS...",1
1,"The word ""haiga"", accepted by the RAE The Roya...",1
2,YORDI ROSADO WILL WRITE AND DESIGN THE NEW SEP...,1
3,UNAM will train teachers to pass the Pisa test...,0
4,Alert: they intend to approve school books wit...,1
5,One step closer to taking exams 'online'\nAbou...,0
6,UNAM WILL PERFORM ANTI-DOPING TESTS TO STUDENT...,1
7,* NUMBER * year-old boy prepares to enter coll...,0
8,* NUMBER * words that we thought unacceptable ...,0
9,THEY WILL LIMIT THE TIME OF EXIT IN PHILOSOPHY...,1


In [148]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 7

In [149]:
X_train, df = get_input(english_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_train.shape)

(51233, 1500)


In [150]:
X_test, df = get_input(translated_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_test.shape)

(2571, 1500)


In [151]:
model = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 4, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model

In [152]:
Y_train = english_dataset.label.values
model.fit(X_train, Y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

Epoch 1/7
51233/51233 [==============================] - 1790s 35ms/step - loss: 0.2619 - accuracy: 0.9313
Epoch 2/7
51233/51233 [==============================] - 1552s 30ms/step - loss: 0.1451 - accuracy: 0.9584
Epoch 3/7
51233/51233 [==============================] - 1713s 33ms/step - loss: 0.1162 - accuracy: 0.9737
Epoch 4/7
51233/51233 [==============================] - 1955s 38ms/step - loss: 0.1060 - accuracy: 0.9749
Epoch 5/7
51233/51233 [==============================] - 1868s 36ms/step - loss: 0.0840 - accuracy: 0.9817
Epoch 6/7
51233/51233 [==============================] - 1677s 33ms/step - loss: 0.0622 - accuracy: 0.9876
Epoch 7/7
51233/51233 [==============================] - 1589s 31ms/step - loss: 0.0716 - accuracy: 0.9839


In [153]:
Y_test = translated_dataset.label.values
loss, acc = model.evaluate(X_test, Y_test)
print(loss, round(acc, 3))

2571/2571 [==============================] - 19s 7ms/step
1.766238255057526 0.532


##### Sharing Tokenizer

In [4]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 7

In [5]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
translated_dataset = pd.read_csv('../data/Merged/spanish_t_dataset.csv')

In [6]:
X_train, X_test, df1, df2 = get_input_share_tokenizer(english_dataset, translated_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_train.shape, X_test.shape)

(51233, 1500) (2571, 1500)


In [7]:
model = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 4, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
Y_train = english_dataset.label.values
model.fit(X_train, Y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit model
Y_test = translated_dataset.label.values
loss, acc = model.evaluate(X_test, Y_test)
print(loss, round(acc, 3))


Epoch 1/7
51233/51233 [==============================] - 1575s 31ms/step - loss: 0.2703 - accuracy: 0.9306
Epoch 2/7
51233/51233 [==============================] - 2050s 40ms/step - loss: 0.1733 - accuracy: 0.9452
Epoch 3/7
51233/51233 [==============================] - 1798s 35ms/step - loss: 0.1365 - accuracy: 0.9631
Epoch 4/7
51233/51233 [==============================] - 1864s 36ms/step - loss: 0.1492 - accuracy: 0.9620
Epoch 5/7
51233/51233 [==============================] - 1755s 34ms/step - loss: 0.1208 - accuracy: 0.9725
Epoch 6/7
51233/51233 [==============================] - 1642s 32ms/step - loss: 0.1090 - accuracy: 0.9763
Epoch 7/7
2571/2571 [==============================] - 15s 6ms/step
1.4473208038318208 0.567
